# initial

In [ ]:
pip install sklearn

In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [26]:
data = pd.read_csv("Darknet.CSV", low_memory=False, parse_dates=["Timestamp"], error_bad_lines=False)

/tmp/ipykernel_2162084/384847805.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("Darknet.CSV", low_memory=False, parse_dates=["Timestamp"], error_bad_lines=False)
Skipping line 328: expected 85 fields, saw 125



In [41]:
def display_all(df): # tip: you can transpose before giving input!
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        print("The shape is: ", data.shape)
        data.info()
        data.dtypes
        display(df)
        

def display_types(df):
    print(df["Level1"].unique())
    print(df["Level2"].unique())
    print(df.groupby("Level1")["Level2"].unique())
    print(df.groupby("Level1")["Level2"].nunique())

In [4]:
# TODO drop VPN
data.drop(["Flow ID"],axis = 1,inplace = True)
data.drop(["Src IP"],axis = 1,inplace = True)
data.drop(["Dst IP"],axis = 1,inplace = True)
data.drop(["Src Port"],axis = 1,inplace = True)
data.drop(["Dst Port"],axis = 1,inplace = True)
data.drop(["Flow Duration"],axis = 1,inplace = True)
data.drop(["Timestamp"],axis = 1,inplace = True)
# data.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
# data = data.loc[:, internet_ds_test.apply(pd.Series.nunique) != 1]
data.isnull().sum()

Protocol                      0
Total Fwd Packet              0
Total Bwd packets             0
Total Length of Fwd Packet    0
Total Length of Bwd Packet    0
                             ..
Idle Std                      0
Idle Max                      0
Idle Min                      0
Label                         0
Label.1                       0
Length: 78, dtype: int64

In [42]:
data.rename(columns = {"Label" : "Level1", "Label.1" : "Level2"}, inplace = True)
# data.rename({'Level2': {"AUDIO-STREAMING" : "Audio-Streaming", "Audio-streaming" : "Audio-Streaming", "Video-streaming" : "Video-Streaming", "File-transfer" : "File-Transfer"}}, inplace = True)
data['Level2'].loc[data['Level2'] == 'AUDIO-STREAMING'] = 'Audio-Streaming'
data['Level2'].loc[data['Level2'] == 'Audio-streaming'] = 'Audio-Streaming'
data['Level2'].loc[data['Level2'] == 'File-transfer'] = 'File-Transfer'
data['Level2'].loc[data['Level2'] == 'Video-streaming'] = 'Video-Streaming'
data["Level2"].unique()
# samples[real_columns] = samples[real_columns].replace([np.inf, -np.inf], np.nan)
# samples[real_columns] = samples[real_columns].dropna()

/tmp/ipykernel_2162084/2898223628.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Level2'].loc[data['Level2'] == 'AUDIO-STREAMING'] = 'Audio-Streaming'
/tmp/ipykernel_2162084/2898223628.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Level2'].loc[data['Level2'] == 'Audio-streaming'] = 'Audio-Streaming'
/tmp/ipykernel_2162084/2898223628.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Level2'].loc[data['Leve

array(['Audio-Streaming', 'Browsing', 'Chat', 'Email', 'File-Transfer',
       'P2P', 'Video-Streaming', 'VOIP'], dtype=object)

# Preprocessing

In [ ]:
display_all(data.tail())

In [11]:
start_mem = data.memory_usage().sum() / 1024**2 # start mem for comparison later

for col in data.columns:
    col_type = data[col].dtypes
    if col_type in ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']:
        c_min = data[col].min()
        c_max = data[col].max()
        if str(col_type)[:3] == 'int': # if it's int
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max: # if it can be int8, make it int8
                data[col] = data[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max: # if it can be int16, make it int16
                data[col] = data[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max: # so on
                data[col] = data[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                data[col] = data[col].astype(np.int64)
        else: # if it's float
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max: # if it can be float16, make it float16
                data[col] = data[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max: # so on
                data[col] = data[col].astype(np.float32)
            else:
                data[col] = data[col].astype(np.float64)

end_mem = data.memory_usage().sum() / 1024**2 # end mem for comparison later

print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

Memory usage after optimization is: 31.31 MB
Decreased by 60.7%


In [ ]:
display_all(data.tail())

In [ ]:
# scaler = StandardScaler().fit(model_samples[real_columns])
# model_samples[real_columns] = scaler.transform(model_samples[real_columns])
# model_samples[real_columns].head()
# samples = samples.dropna()
# model_samples = model_samples.dropna()
# print(samples[samples.columns[samples.isna().sum()>0]].isna().sum())

# Split

In [63]:
copied = data.copy()

In [77]:
Y = copied[["Level1", "Level2"]]
X = copied.drop(["Level1", "Level2"], axis=1)

In [76]:
X

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,2015-07-24 16:09:48,229,1,1,...,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,10.152.152.11-216.58.220.99-57159-443-6,10.152.152.11,57159,216.58.220.99,443,6,2015-07-24 16:09:48,407,1,1,...,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,10.152.152.11-216.58.220.99-57160-443-6,10.152.152.11,57160,216.58.220.99,443,6,2015-07-24 16:09:48,431,1,1,...,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,10.152.152.11-74.125.136.120-49134-443-6,10.152.152.11,49134,74.125.136.120,443,6,2015-07-24 16:09:48,359,1,1,...,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,10.152.152.11-173.194.65.127-34697-19305-6,10.152.152.11,34697,173.194.65.127,19305,6,2015-07-24 16:09:45,10778451,591,400,...,581,20,0,0,0,0,1.437765e+15,3.117718e+06,1.437765e+15,1.437765e+15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141525,10.8.8.246-224.0.0.252-55219-5355-17,10.8.8.246,55219,224.0.0.252,5355,17,2015-05-22 13:55:03,411806,2,0,...,1,8,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
141526,10.8.8.246-224.0.0.252-64207-5355-17,10.8.8.246,64207,224.0.0.252,5355,17,2015-05-22 14:09:05,411574,2,0,...,1,8,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
141527,10.8.8.246-224.0.0.252-61115-5355-17,10.8.8.246,61115,224.0.0.252,5355,17,2015-05-22 14:19:31,422299,2,0,...,1,8,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
141528,10.8.8.246-224.0.0.252-64790-5355-17,10.8.8.246,64790,224.0.0.252,5355,17,2015-05-22 14:29:55,411855,2,0,...,1,8,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [ ]:
# FEATURE EXTRACTION

# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# count_vect = CountVectorizer(stop_words='english')
# transformer = TfidfTransformer(sublinear_tf=True)

# X_train_cnt = count_vect.fit_transform(X_train)
# X_train_TF = transformer.fit_transform(X_train_cnt)
# print(X_train_cnt.shape)
# print(X_train_TF.shape)

# X_test_cnt = count_vect.transform(X_test)
# X_test_TF = transformer.transform(X_test_cnt)
# print(X_test_cnt.shape)
# print(X_test_TF.shape)

In [85]:
from sklearn.ensemble import RandomForestClassifier as RFClass
ranForModel = RFClass(n_estimators=10, criterion = "entropy")
ranForModel.fit(X_train_TF, Y_train)

NameError: name 'X_train_TF' is not defined